### Edgar Moises Hernandez-Gonzalez
#### 05/06/20-01/07/20
#### Clasificacion de imagenes motoras en EEG con CNN-2D
##### Caracteristicas = Espectrogramas STFFT (3D)
##### Entrada de 990x31

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import time
from scipy.signal import spectrogram
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

Using TensorFlow backend.


In [ ]:
# para que este script funcione debe de decir 'channels_last', si dice 'channels_first' no sirve
K.image_data_format()

'channels_last'

In [ ]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/Datos/MI-EEG-A06T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/Datos/MI-EEG-A06E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/Datos/EtiquetasA06T.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/Datos/EtiquetasA06E.csv", header=None)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 22000)
(288, 22000)
(288, 1)
(288, 1)


In [ ]:
x = pd.concat([x_train, x_test])
y = pd.concat([y_train, y_test])

In [ ]:
print(x.shape)
print(y.shape)

(576, 22000)
(576, 1)


In [ ]:
# en caso de no tener el y_test se puede utilizar el x_test
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 576


In [ ]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y))

print("n_classes:", n_classes)

n_classes: 4


In [ ]:
# codificar etiquetas con valores de 0 a nClases-1
y = LabelEncoder().fit_transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#unir 22 canales y resize de tamaño proporcional
def unir_espectrogramas_vertical(data):
  fs = 250
  datos = np.zeros((data.shape[0],990,31))
  temporal = np.zeros((990,31))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      #espectrograma de STFFT
      f, t, Sxx = spectrogram(sig, fs=250, window='hann', nperseg=250,
                              noverlap=225, nfft=500, scaling='spectrum')
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [ ]:
#llamar a unir_espectrogramas_vertical_resize(data)
inicio = time.time()

x = unir_espectrogramas_vertical(x)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
400
500
Tiempo: 13.400630474090576


In [ ]:
print(x.shape)

(576, 990, 11)


In [ ]:
print(np.max(x))
print(np.min(x))
print(np.mean(x))

452.9424916745041
2.426135596687323e-09
0.9039499415108267


In [ ]:
#convertir a float
x = x.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x /= np.ceil(np.max(x))

In [ ]:
print(np.max(x))
print(np.min(x))
print(np.mean(x))

0.9998731
5.3557076e-12
0.0019954739


In [ ]:
#convertir de 3D a 4D
x = x.reshape((x.shape[0], x.shape[1], x.shape[2], 1))

print(x.shape)

(576, 990, 11, 1)


In [ ]:
def CNN_2D(n_filtros, tamano_filtros, n_neuronas):
  model = Sequential()
  model.add(Conv2D(n_filtros, tamano_filtros, activation='relu', padding='same',
                   input_shape=x.shape[1:]))
  model.add(MaxPool2D((2,2)))
  model.add(Conv2D(n_filtros, tamano_filtros, activation='relu', padding='same'))
  model.add(MaxPool2D((2,2)))
  model.add(Flatten())
  model.add(Dense(n_neuronas, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(n_classes, activation='softmax'))

  optimizer = Adam(lr=1e-4)
  model.compile(optimizer = optimizer,
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [ ]:
clf = KerasClassifier(build_fn = CNN_2D, epochs=400, batch_size=36, verbose=0)

In [ ]:
n_filtros = [4, 8, 16]
tamano_filtros = [(3,3), (15,3)]
n_neuronas = [32, 64, 128]
hiperparametros = dict(n_filtros=n_filtros,
                       tamano_filtros=tamano_filtros,
                       n_neuronas=n_neuronas)

In [ ]:
# 48 redes tardan 2474seg
# entonces 32 redes = 1649
# entonces 24 redes = 1021
# entonces 18 redes = 780
3*2*3

18

In [ ]:
grid = GridSearchCV(clf, hiperparametros, cv=2)

In [ ]:
inicio = time.time()

grid.fit(x, y)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 797.6594841480255


In [ ]:
print(grid.best_params_)
print(grid.best_score_)

{'n_filtros': 4, 'n_neuronas': 64, 'tamano_filtros': (3, 3)}
0.4704861044883728


In [ ]:
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

0.464 (+/-0.017) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.439 (+/-0.031) for {'n_filtros': 4, 'n_neuronas': 32, 'tamano_filtros': (15, 3)}
0.470 (+/-0.003) for {'n_filtros': 4, 'n_neuronas': 64, 'tamano_filtros': (3, 3)}
0.439 (+/-0.010) for {'n_filtros': 4, 'n_neuronas': 64, 'tamano_filtros': (15, 3)}
0.448 (+/-0.014) for {'n_filtros': 4, 'n_neuronas': 128, 'tamano_filtros': (3, 3)}
0.429 (+/-0.010) for {'n_filtros': 4, 'n_neuronas': 128, 'tamano_filtros': (15, 3)}
0.460 (+/-0.017) for {'n_filtros': 8, 'n_neuronas': 32, 'tamano_filtros': (3, 3)}
0.443 (+/-0.010) for {'n_filtros': 8, 'n_neuronas': 32, 'tamano_filtros': (15, 3)}
0.450 (+/-0.003) for {'n_filtros': 8, 'n_neuronas': 64, 'tamano_filtros': (3, 3)}
0.431 (+/-0.007) for {'n_filtros': 8, 'n_neuronas': 64, 'tamano_filtros': (15, 3)}
0.443 (+/-0.017) for {'n_filtros': 8, 'n_neuronas': 128, 'tamano_filtros': (3, 3)}
0.422 (+/-0.003) for {'n_filtros': 8, 'n_neuronas': 128, 'tamano_filtros': (15, 3)}
0.443 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jul  1 16:28:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    41W / 250W |   1587MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------